Kimonet demo
------------

Install necessary libraries

In [ ]:
!apt-get install libgraphviz-dev
!pip install kimonet

This section defines the states that will play a role in the simulation.
Each state is defined by a unique label and an energy in eV.
The ground state (GS) is special. GS never transport or decays and it is used
as a background state in which all molecules that are not part of another state are.
All GS are equal and are always bounded to a single molecule.
For the purpose of transport definitions GS can be imported from kimonet module
as shown below.

In this example state 's1' is defined in variable s1 and GS is imported as gs variable

In [ ]:
from kimonet.system.state import ground_state as gs
from kimonet.system.state import State

s1 = State(label='s1', energy=20.0)

Here we define state pairs properties. All properties that involve two states
are defined by Transition object. Al list of Transition objects in needed as argument
for some of the functions defined later on. Here only one transition is defined between
states s1 and gs.

In [ ]:
from kimonet.core.processes.transitions import Transition

transitions = [Transition(s1, gs,  # the pair of states
                          tdm=[0.1, 0.0],  # a.u.
                          reorganization_energy=0.08)] # eV

In this section the system to be simulated is defined using *crystal_system* function.
This function created a crystal from two Molecule type objects.
Molecule objects contain information about the site. In this case each molecule
has a different site energy (in eV).
*crystal_system* function uses these two molecules as reference and creates a
supercell with copies of these molecules placed in the positions according to
scaled_site_coordinates and unitcell.
- unitcell defines the lattice vectors (rows) in angstroms
- scaled_site_coordinates defines the sites in crystallographic coordinates (columns)
of each molecule type (rows).
- dimensions defined the size (in unitcells) of the whole supercell.
- Orientation defines the orientation of each of the molecule types as rotations respect to axis x, y and z (in this order).

The dimensionality of the system is given by the length of the vectors given in scaled_site_coordinates/unitcell.
Orientation is always a 3D vector.
By default all molecules are bound to ground states.


In [ ]:
from kimonet.system.molecule import Molecule
from kimonet.system.generators import crystal_system

# define system as a crystal
molecule_type_1 = Molecule(site_energy=0.0)  # eV
molecule_type_2 = Molecule(site_energy=0.5)  # eV

system = crystal_system(molecules=[molecule_type_1, molecule_type_2],
                        scaled_site_coordinates=[[0.0, 0.0],
                                                 [0.0, 0.5]],
                        unitcell=[[5.0, 1.0],
                                  [1.0, 5.0]],
                        dimensions=[2, 2],  # supercell size
                        orientations=[[0.0, 0.0, 1.5708],  # Rx, Ry, Rz
                                      [0.0, 0.0, 0.0]])    # Rx, Ry, Rz

Here the exctions are placed on the molecules. In this example 0 and 1 correspond to molecule
indices in the system. Using visualization functions is possible to check which molecules are these exactly.

In [ ]:
system.add_excitation_index(s1, 0)
system.add_excitation_index(s1, 1)

In this section the processes that take place in the simulation are defined. These objects makes use
of the states and transitions defined above. For this example two processes are defined:
1) a transport process and 2) a decay for state s1.

In [ ]:
from kimonet.system.vibrations import MarcusModel
from kimonet.core.processes.couplings import forster_coupling
from kimonet.core.processes.decays import einstein_radiative_decay
from kimonet.core.processes.types import GoldenRule, DecayRate


system.process_scheme = [GoldenRule(initial_states=(s1, gs), final_states=(gs, s1),
                                    electronic_coupling_function=forster_coupling,
                                    description='Forster coupling',
                                    arguments={'ref_index': 1,
                                               'transitions': transitions},
                                    vibrations=MarcusModel(transitions=transitions) # eV
                                    ),
                        DecayRate(initial_state=s1, final_state=gs,
                                  decay_rate_function=einstein_radiative_decay,
                                  arguments={'transitions': transitions},
                                  description='custom decay rate')
                        ]

Set additional parameters of the simulation

In [ ]:
system.cutoff_radius = 8  # interaction cutoff radius in Angstrom


The unit cell can be visualized using visualize_system

In [ ]:
from kimonet.analysis import visualize_system

visualize_system(system)

To make sure that the processes are well defined it is useful to list all the processes
for the initial configuration of the system. This can be done by system_test_info function.

In [ ]:
from kimonet import system_test_info

system_test_info(system)


Run Kinetic MonteCarlo algorithm.
This function calculates several kMC trajectories from the initial configuration.
max_steps is used to truncate the number of steps of each trajectory. Ideally
the simulation will finish when reaching a configuration in which no processes can occur,
however in certain process schemes this may lead to never ending simulations. To avoid this
max_steps acts as a security measure. If the trajectory is truncated a warning will be risen.

This function returns a list of trajectories that can be analyzed later.

In [ ]:
from kimonet import calculate_kmc

trajectories = calculate_kmc(system,
                             num_trajectories=5,    # number of trajectories that will be simulated
                             max_steps=100,        # maximum number of steps for trajectory allowed
                             silent=True)

This section displays the results of the simulation. For this TrajectoryAnalysis object is defined.
This object contains the methods to calculate the different results.

In [ ]:
from kimonet.analysis import TrajectoryAnalysis

analysis = TrajectoryAnalysis(trajectories)

print('diffusion coefficient: {:9.5e} Angs^2/ns'.format(analysis.diffusion_coefficient()))
print('lifetime:              {:9.5e} ns'.format(analysis.lifetime()))
print('diffusion length:      {:9.5e} Angs'.format(analysis.diffusion_length()))
print('diffusion tensor (angs^2/ns)')



The simulation results calculated per state.

In [ ]:
for state in analysis.get_states():
    print('\nState: {}\n--------------------------------'.format(state))
    print('diffusion coefficient: {:9.5e} Angs^2/ns'.format(analysis.diffusion_coefficient(state)))
    print('lifetime:              {:9.5e} ns'.format(analysis.lifetime(state)))
    print('diffusion length:      {:9.5e} Angs'.format(analysis.diffusion_length(state)))
    print('diffusion tensor (angs^2/ns)')

Plot diffusion coefficient and length tensors.

In [ ]:
from kimonet.analysis import plot_polar_plot

plot_polar_plot(analysis.diffusion_coeff_tensor('s1'), 
                title='Diffusion', 
                plane=[0, 1])

plot_polar_plot(analysis.diffusion_length_square_tensor('s1', unit_cell=[[5.0, 1.0],
                                                                         [1.0, 5.0]]),
                title='Diffusion length square', 
                crystal_labels=True, 
                plane=[0, 1])


Plot exciton density distances.

In [ ]:
for state in analysis.states:
    analysis.plot_exciton_density(state)
plt = analysis.plot_exciton_density()
plt.show()

Plot exciton trajectories.

In [ ]:
analysis.plot_2d('s1').show()